In [1]:
import os 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

/Users/pranathirudraraju/Desktop/Projects/Chatbot/Muti-tool-agent/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv('../../../../.env')

True

In [6]:
api_key = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",  # or gemini-1.5-flash for faster/cheaper
    google_api_key=api_key
)

In [7]:
response = llm.invoke("Write a short real estate property description for a 3BHK in Seattle")
print(response.content)

Of course! Here are a few options for a short real estate property description for a 3-bedroom home in Seattle, each with a slightly different tone.

*(Note: In the US market, "3BHK" is described as a "3 Bedroom" or "3 Bed, 2 Bath" home.)*

---

### Option 1: Modern & Punchy

**Bright & Spacious 3-Bedroom Seattle Gem!**

Your Seattle story starts here! This beautifully maintained 3-bedroom, 2-bathroom home offers the perfect blend of comfort and style. The open-concept living area is bathed in natural light, creating an inviting atmosphere for relaxing or entertaining. A modern kitchen awaits with stainless steel appliances and ample counter space. The primary suite provides a peaceful retreat with its own private bath, while two additional bedrooms offer fantastic flexibility for a home office, guests, or family.

*   **Highlights:** 3 Spacious Bedrooms, 2 Full Baths, open layout, private balcony for morning coffee, and dedicated parking.
*   **Location:** Nestled in a vibrant neighbo

In [ ]:
import os
import requests
from dotenv import load_dotenv

# -------------------- Load Environment --------------------
load_dotenv('../../../../.env')  # Adjust path to your .env
RAPIDAPI_KEY = os.getenv("RAPIDAPI_KEY")

if not RAPIDAPI_KEY:
    raise ValueError("❌ RAPIDAPI_KEY not found in .env")

# -------------------- Helper Functions --------------------
def get_property_images(zpid):
    """Fetch images of a property using its Zillow zpid"""
    url = "https://zillow-com1.p.rapidapi.com/images"
    querystring = {"zpid": zpid}

    headers = {
        "x-rapidapi-host": "zillow-com1.p.rapidapi.com",
        "x-rapidapi-key": RAPIDAPI_KEY
    }

    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code != 200:
        print("❌ Failed to fetch images:", response.status_code)
        return []

    data = response.json()
    return data.get("images", [])

def get_property_data(address, citystatezip):
    """
    Fetch property details and images from Zillow API
    :param address: Street address (ex: "2114 Bigelow Ave N")
    :param citystatezip: City + State + ZIP (ex: "Seattle, WA 98109")
    """
    url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"
    querystring = {"location": f"{address}, {citystatezip}"}

    headers = {
        "x-rapidapi-host": "zillow-com1.p.rapidapi.com",
        "x-rapidapi-key": RAPIDAPI_KEY
    }

    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code != 200:
        print("❌ Error fetching property:", response.status_code, response.text)
        return None

    data = response.json()
    # Debug: show full API response
    # print("DEBUG: Full API response:", data)

    props = data.get("props")
    if not props or len(props) == 0:
        print("❌ No property found for this address")
        return None

    # Take first property (best match)
    property_info = props[0]

    zpid = property_info.get("zpid")
    if zpid:
        images = get_property_images(zpid)
    else:
        images = []

    property_info["images"] = images
    return property_info

# -------------------- Run Example --------------------
if __name__ == "__main__":
    # Input address
    street_address = "46 Creekstone Ln"
    city_state_zip = "Dawsonville, GA 30534"

    # Fetch property data
    result = get_property_data(street_address, city_state_zip)

    if not result:
        print("❌ Could not fetch property data. Please check the address or Zillow availability.")
    else:
        print("\n🏠 Property Info:")
        print("Address:", result.get("address"))
        print("Price:", result.get("price"))
        print("Bedrooms:", result.get("bedrooms"))
        print("Bathrooms:", result.get("bathrooms"))
        print("Living Area:", result.get("livingArea"))
        print("Year Built:", result.get("yearBuilt"))

        print("\n📸 Images (first 5):")
        for img in result["images"][:5]:
            print(img)


Address: 2276 La Granada Dr, Los Angeles, CA 90068
Price: 2888000
Bedrooms: 4
Bathrooms: 3
Year Built: None
ZPID: 20804151
------------------------------
Address: 8363 Stewart Ave, Los Angeles, CA 90045
Price: 4195000
Bedrooms: 5
Bathrooms: 7
Year Built: None
ZPID: 20383717
------------------------------


In [4]:
import os
import requests

RAPIDAPI_KEY = "efa9c09e8emsh8aa7f352d1e23e3p136dffjsnd15ed3600b48"  # Or load from .env

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {
    "location": "46 Creekstone Ln, Dawsonville, GA 30534",
    "status_type": "ForSale",
    "home_type": "Houses"
}

headers = {
    "x-rapidapi-host": "zillow-com1.p.rapidapi.com",
    "x-rapidapi-key": RAPIDAPI_KEY
}

response = requests.get(url, headers=headers, params=querystring)

if response.status_code != 200:
    print("❌ Error:", response.status_code, response.text)
else:
    data = response.json()
    # Pretty print first 2 properties
    for prop in data.get("props", [])[:2]:
        print("Address:", prop.get("address"))
        print("Price:", prop.get("price"))
        print("Bedrooms:", prop.get("bedrooms"))
        print("Bathrooms:", prop.get("bathrooms"))
        print("Year Built:", prop.get("yearBuilt"))
        print("ZPID:", prop.get("zpid"))
        print("-" * 30)


In [5]:
import requests

RAPIDAPI_KEY = "efa9c09e8emsh8aa7f352d1e23e3p136dffjsnd15ed3600b48"

def get_property_by_address(full_address, citystatezip):
    url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"
    querystring = {
        "location": citystatezip,
        "status_type": "ForSale",
        "home_type": "Houses"
    }

    headers = {
        "x-rapidapi-host": "zillow-com1.p.rapidapi.com",
        "x-rapidapi-key": RAPIDAPI_KEY
    }

    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code != 200:
        print("❌ Error:", response.status_code, response.text)
        return None

    data = response.json()
    for prop in data.get("props", []):
        if full_address.lower() in prop.get("address", "").lower():
            return prop

    print("❌ Property not found in results")
    return None


# ---------- Run Example ----------
full_address = "46 Creekstone Ln"
citystatezip = "Dawsonville, GA 30534"

result = get_property_by_address(full_address, citystatezip)
if result:
    print("🏠 Property Info:")
    print("Address:", result.get("address"))
    print("Price:", result.get("price"))
    print("Bedrooms:", result.get("bedrooms"))
    print("Bathrooms:", result.get("bathrooms"))
    print("Year Built:", result.get("yearBuilt"))
    print("ZPID:", result.get("zpid"))


🏠 Property Info:
Address: 46 Creekstone Ln, Dawsonville, GA 30534
Price: 435000
Bedrooms: 4
Bathrooms: 3
Year Built: None
ZPID: 243372527


In [6]:
import requests

RAPIDAPI_KEY = "efa9c09e8emsh8aa7f352d1e23e3p136dffjsnd15ed3600b48"

HEADERS = {
    "x-rapidapi-host": "zillow-com1.p.rapidapi.com",
    "x-rapidapi-key": RAPIDAPI_KEY
}

def get_full_property_info(zpid):
    # Property details
    url = f"https://zillow-com1.p.rapidapi.com/property"
    params = {"zpid": zpid}
    resp = requests.get(url, headers=HEADERS, params=params)
    if resp.status_code != 200:
        print("❌ Failed to get property details")
        return None
    details = resp.json()

    # Images
    url_img = f"https://zillow-com1.p.rapidapi.com/images"
    resp_img = requests.get(url_img, headers=HEADERS, params={"zpid": zpid})
    images = resp_img.json().get("images", [])

    details["images"] = images
    return details


# Example usage:
zpid = 243372527
full_info = get_full_property_info(zpid)

if full_info:
    print("🏠 Full Property Info:")
    print("Address:", full_info.get("address"))
    print("Price:", full_info.get("price"))
    print("Bedrooms:", full_info.get("bedrooms"))
    print("Bathrooms:", full_info.get("bathrooms"))
    print("Year Built:", full_info.get("yearBuilt"))
    print("Lot Size:", full_info.get("lotSize"))
    print("Living Area:", full_info.get("livingArea"))
    print("ZPID:", full_info.get("zpid"))
    print("\n📸 Images:")
    for img in full_info["images"][:5]:
        print(img)


🏠 Full Property Info:
Address: {'community': None, 'city': 'Dawsonville', 'state': 'GA', 'neighborhood': None, 'subdivision': 'Creekstone', 'streetAddress': '46 Creekstone Ln', 'zipcode': '30534'}
Price: 435000
Bedrooms: 4
Bathrooms: 3
Year Built: 2020
Lot Size: None
Living Area: 2252
ZPID: 243372527

📸 Images:
https://photos.zillowstatic.com/fp/19d17e6055480dc2ccf9858b932a3ebb-cc_ft_1536.jpg
https://photos.zillowstatic.com/fp/2ce7a8e6b84d2edd6e84f3bb343eae75-cc_ft_1536.jpg
https://photos.zillowstatic.com/fp/656101ba0888abdfe7614a6f01944d1b-cc_ft_1536.jpg
https://photos.zillowstatic.com/fp/231c04e05c0f6a8470ed060fe22da930-cc_ft_1536.jpg
https://photos.zillowstatic.com/fp/acb431e78770fd2be75f5ac85ab1925f-cc_ft_1536.jpg


In [10]:
import json
with open(f"property_{zpid}.json", "w") as f:
        json.dump(full_info, f, indent=4)
    
print(f"\n✅ Property data saved to property_{zpid}.json")


✅ Property data saved to property_243372527.json


In [18]:
import json

def recursive_chunk_safe(data, max_chunk_size, meta_stack=None):
    if meta_stack is None:
        meta_stack = []

    result_chunks = []

    def helper(d, meta_stack):
        # Leaf node
        if not isinstance(d, (dict, list)):
            chunk = {"meta": list(meta_stack), "value": d}
            return [chunk]

        chunks = []

        if isinstance(d, dict):
            current_chunk = {}
            for k, v in d.items():
                meta_stack.append(k)
                sub_chunks = helper(v, meta_stack)
                meta_stack.pop()
                
                for sub in sub_chunks:
                    temp_chunk = current_chunk.copy()
                    temp_chunk[k] = sub
                    if len(json.dumps(temp_chunk)) > max_chunk_size:
                        if current_chunk:
                            chunks.append(current_chunk)
                        current_chunk = {k: sub}
                    else:
                        current_chunk[k] = sub
            if current_chunk:
                chunks.append(current_chunk)

        elif isinstance(d, list):
            current_list = []
            for item in d:
                sub_chunks = helper(item, meta_stack)
                for sub in sub_chunks:
                    temp_list = current_list + [sub]
                    if len(json.dumps(temp_list)) > max_chunk_size:
                        if current_list:
                            chunks.append(current_list)
                        current_list = [sub]
                    else:
                        current_list.append(sub)
            if current_list:
                chunks.append(current_list)

        return chunks

    # wrap each chunk with meta for top level
    top_level_chunks = helper(data, meta_stack)
    for c in top_level_chunks:
        result_chunks.append({"meta": list(meta_stack), "data": c})

    return result_chunks

# Example usage
with open("property_243372527.json") as f:
    data = json.load(f)

max_size = 1000  # max chunk size in characters
chunks = recursive_chunk_safe(data, max_size)

# # Save chunks
# for idx, chunk in enumerate(chunks):
#     with open(f"chunk_{idx}.json", "w") as out:
#         json.dump(chunk, out, indent=2)

for idx, chunk in enumerate(chunks):
    print(chunk)


{'meta': [], 'data': {'buildingPermits': {'meta': ['buildingPermits'], 'value': None}}}
{'meta': [], 'data': {'contact_recipients': [{'zuid': {'meta': ['contact_recipients', 'zuid'], 'value': 'X1-ZUv2ktj4jt3nyh_98wrm'}, 'image_url': {'meta': ['contact_recipients', 'image_url'], 'value': 'https://www.zillowstatic.com/static/images/nophoto_h_g.png'}}], 'longitude': {'meta': ['longitude'], 'value': -84.13145}, 'countyFIPS': {'meta': ['countyFIPS'], 'value': '13085'}, 'imgSrc': {'meta': ['imgSrc'], 'value': 'https://photos.zillowstatic.com/fp/19d17e6055480dc2ccf9858b932a3ebb-p_d.jpg'}, 'livingAreaValue': {'meta': ['livingAreaValue'], 'value': 2252}, 'streetAddress': {'meta': ['streetAddress'], 'value': '46 Creekstone Ln'}, 'county': {'meta': ['county'], 'value': 'Dawson County'}, 'monthlyHoaFee': {'meta': ['monthlyHoaFee'], 'value': 44}, 'timeZone': {'meta': ['timeZone'], 'value': 'America/New_York'}}}
{'meta': [], 'data': {'taxHistory': [{'time': {'meta': ['taxHistory', 'time'], 'value': 

In [27]:
import os
import json
import re
from langchain_google_genai import ChatGoogleGenerativeAI

# ------------------- Load property JSON -------------------
zpid = 243372527
with open(f"property_{zpid}.json", "r") as f:
    property_data = json.load(f)

images = property_data.get("images", [])

# ------------------- Initialize Gemini LLM -------------------
api_key = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=api_key
)

# ------------------- Define Fields -------------------
fields = [
    "Roof Type",
    "Exterior Material",
    "Pool",
    "Garage",
    "Number of Stories",
    "General Condition / Renovation Indicators",
    "Lot Size / Backyard Area",
    "Driveway Type / Paved Area",
    "Solar Panels / External Installations"
]

# ------------------- Construct Prompt -------------------
prompt = f"""
You are an expert home underwriter AI. Analyze the following property images.
Fill ONLY the following fields in STRICT JSON format (no reasoning, no extra text):

Fields: {', '.join(fields)}

Images: {images}

Example output:
{{
"Roof Type": "Asphalt Shingle",
"Exterior Material": "Brick",
"Pool": "Yes",
"Garage": "2-car",
"Number of Stories": "2",
"General Condition / Renovation Indicators": "Good",
"Lot Size / Backyard Area": "Medium",
"Driveway Type / Paved Area": "Concrete",
"Solar Panels / External Installations": "None"
}}
"""

# ------------------- Call Gemini -------------------
response = llm.predict(prompt)

# ------------------- Clean & Parse JSON -------------------
# Remove Markdown or extra characters
cleaned_response = re.sub(r"^```json|```$", "", response.strip())

try:
    image_analysis_json = json.loads(cleaned_response)
except json.JSONDecodeError:
    print("❌ Failed to parse JSON from LLM output:")
    print(response)
    image_analysis_json = None

# ------------------- Output -------------------
if image_analysis_json:
    print("✅ Image Analysis Result:")
    print(json.dumps(image_analysis_json, indent=4))
    # Optional: Save to file
    with open(f"property_{zpid}_image_analysis.json", "w") as f:
        json.dump(image_analysis_json, f, indent=4)
else:
    print("⚠️ No valid JSON returned from Gemini.")


✅ Image Analysis Result:
{
    "Roof Type": "Asphalt Shingle",
    "Exterior Material": "Brick and Siding",
    "Pool": "Yes",
    "Garage": "Attached 2-car",
    "Number of Stories": "2",
    "General Condition / Renovation Indicators": "Excellent; Renovated kitchen and bathrooms",
    "Lot Size / Backyard Area": "Large",
    "Driveway Type / Paved Area": "Concrete",
    "Solar Panels / External Installations": "None"
}
